# Векторные представления речи Wav2Vec

Идеи сети BERT, которая создает векторные представления для текста применима и для речи. Так работает сеть Wav2Vec.

Она работает в двух фазах:
- А. Самообучение векторному представлению речи,
- Б. Обучение с учителем предсказанию символов с ошибкой CTC (см. прошлую тему).

![img](https://cdn.neurosys.com/wp-content/uploads/2021/09/1_training-phases-of-wav2vec-2.0.png.webp)


![img](https://cdn.neurosys.com/wp-content/webp-express/webp-images/uploads/2021/09/2_results-of-wav2vec-2.0.png.webp)




## А. Самообучение векторному представлению речи.
![img](https://cdn.neurosys.com/wp-content/webp-express/webp-images/uploads/2021/09/4_wav2vec-2.0-model-architecture-1536x765.png.webp)


Временно`е представление звука **X** пропускается через сверточные слои, которые создают латентное представление **Z**. Оно поступает на сеть Transformer (кодер, в котором используется механизм внимания), где переводится в контекстное представление **C**.

Из представления **Z** создают его квантованное представление **Q**.

Часть токенов (здесь временных отсчетов), как и в BERT, маскируется, а сеть учится восстанавливать из маскированного представления **Z** квантованное **Q** (т.е. **Q** есть цель для **C**). Это самообучение, так как здесь не нужны размеченные данные, а только сами звуки.



### Квантование
Истинная цель квантования - разбить звук на участки, соответствующие символам, сказать, ага **Z** был такой - значит символы такие, или **Z** был сякой - значит символы сякие. Но, конечно, мы изначально не знаем этого.

Чтобы с имитировать такой процесс сделали несколько обучаемых кодовых книг G, в которых содержатся кодовые вектора Vg. Вектор  **Z** сравнивается со всеми кодовыми векторами и для каждой книги выбирается наиболее похожий Vg. Отобранные вектора из книг контактенируются и собираются в один с помощью линейного слоя (тоже обучаемого). Отбор ведется с помощью измененного и рандомизированного softmax (а именно Gumbel softmax).



![img](https://cdn.neurosys.com/wp-content/uploads/2021/09/6_quantization.png.webp)

![img](https://cdn.neurosys.com/wp-content/uploads/2021/09/7_gumbel-softmax.png.webp)

Не смотрите на сложную формулу, в целом это тот же softmax с изменениями:
- будем считать косинусную похожесть sim между векторами z и v.
- добавим шум n
- добавим параметр $\tau$ - "температуру", который регулирует "крутость" максимума. При $\tau \rightarrow 0$ результат выродится в обычый выбор максимума - one-hot вектор.

Интуитивно процесс можно представить так, что мы вписываем в кодовые книги такие вектора, чтобы для как можно большего количества векторов Z нашлись похожие на них. В пределе в книгах оказались бы сами вектора Z (они похожи сами на себя), но размер книг ограничен и есть шум, что заставляет искать (обучать) вектора Vg удовлетворяющие большему количеству Z.




## Маскирование
Процесс маскировки прост:
- случайно, с заданной вероятностью **p**, отобрать вектора **Z**, которые будут означать начало маски.
- замаскировать **M** векторов следующих последовательно за начальными. Области маскировки могут пересекаться.
- Контрастивная ошибка будет считаться только для центрального отсчета в маске.

![img](https://cdn.neurosys.com/wp-content/uploads/2021/09/9_masking-m-consecutive-time-steps-1536x574.png.webp)


### Функция ошибки


Функция ошибки для обучения состоит из двух частей:
- контрастивная ошибка для маски и
- ошибка разнообразия

В **контрастивной ошибке** сравнивают текущий вектор контекста **Сt** с положительной целью **Qt** и отрицательными целями $\hat Q_t$, взятыми случайно из других отсчетов маски. Цель обучения сделать так, чтобы **Сt** совпадал с **Qt** и не совпадал с $\hat Q_t$. Это можно сделать с помощью функции похожей на softmax

![img](https://cdn.neurosys.com/wp-content/uploads/2021/09/11_contrastive-loss.png.webp)

**Ошибка разнообразия** оценивает, насколько разнообразны кодовые вектора в кодовых книгах, считая их энтропию. Нечто подобное мы уже делали, когда обучали деревья решений, вспомните, там оценивалась энтропия подмножеств в узлах. Цель здесь - увеличить разнообразие векторов, чтобы они не стали все одинаковыми.

![img](https://cdn.neurosys.com/wp-content/uploads/2021/09/12_diversity-loss.png.webp)



В результате сеть самообучается создавать такие контекстные представления **C**, чтобы быть способной предсказать замаскированные квантованные представления **Q**.
   

## Б. Обучение с учителем.
Самообучение только создает новое векторное представление для речи в контекстных векторах **C**.

Для распознавания речи используют все ту же ошибку CTC, а чтобы ее посчитать, в сеть добавляют  полносвязный слой, который из контекстных векторов возвращает уровни уверенности в символах.

В этой фазе квантование уже не нужно, ведь оно лишь создавало цели для самообучения.

![img](https://cdn.neurosys.com/wp-content/webp-express/webp-images/uploads/2021/09/3_fine-tuned-wav2vec-2.0-model-architecture-1536x691.png.webp)

Важно отметить, что самообучение позволяет создавать векторные представления для речи, которые потом можно использовать для классификации или других задач.



# Пример распознавания речи
Реализация Wav2Vec (как и других сетей на основе Transformers) сделана в библиотеке 🤗 [Transformers](https://huggingface.co/transformers/), посмотрим пример использования.

In [ ]:
# для совместимости поставим предыдушую версию
#!pip -q install imageio==2.4.1

In [1]:
# устанавливаем
# трансформеры
!pip  -q install transformers
# загрузка из Youtube
!pip -q install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.4 MB/s eta 0:00:00


In [2]:
# подключаем
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC
from IPython.display import Audio

import moviepy.editor as mp
import torch
import librosa
import os

In [3]:
# загрузим видео
clip = "https://www.youtube.com/watch?v=7Ood-IE7sx4"

# начало и конец распознавания, секунды
start = 0
end = 20

In [4]:
# скачиваем и переименовываем
os.system('yt-dlp {} -v --recode-video mp4'.format(clip))
os.system('mv *.mp4 clip.mp4')

0

Извлекаем аудио, будем делать это в 10-секундных нарезках, чтобы сэкономить память.

In [5]:
clip = mp.VideoFileClip("clip.mp4")
end = min(clip.duration, end)

#
clip_paths = []

# извлекаем аудио
for i in range(start, int(end), 10):
  sub_end = min(i+10, end)
  sub_clip = clip.subclip(i,sub_end) # нарезка

  sub_clip.audio.write_audiofile("audio_" + str(i) + ".mp3") # записываем аудио в файл
  clip_paths.append("audio_" + str(i) + ".mp3")

MoviePy - Writing audio in audio_0.mp3


MoviePy - Done.
MoviePy - Writing audio in audio_10.mp3


MoviePy - Done.


In [6]:
# Загружаем предобученный Wav2Vec из huggingface
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
  warnings.warn(



model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

In [7]:
# Прослушаем | громкий звук!
Audio(clip_paths[0])

Распознаем звук каждой 10-секундной нарезки по одной.

In [8]:
cc = ""

for path in clip_paths:
    # загружаем аудио
    input_audio, _ = librosa.load(path,
                                sr=16000)

    # токенизируем (переводим в  PyTorch тензоры)
    input_values = tokenizer(input_audio, return_tensors="pt", padding="longest").input_values

    #
    with torch.no_grad():
      logits = model(input_values).logits # подаем в модель
      predicted_ids = torch.argmax(logits, dim=-1) # выделяем наиболее уверенный токен

    # декодируем в символы
    transcription = tokenizer.batch_decode(predicted_ids)[0]
    cc += transcription + " "

In [9]:
# печатаем результат
print(cc)

IT'S ORGRATIC AN I HAVE THE HIGH GROUND YOU WONDER AS T MAKE MY NOW GOT FRIED WOGEVER 



# Заключение

Число различных моделей распознавания речи велико, постоянно выходят новые и новые. Другие примеры можно посмотреть здесь:

https://colab.research.google.com/github/snakers4/silero-models/blob/master/examples.ipynb#scrollTo=jwKw-yMpDQTL

https://github.com/snakers4/silero-models
https://habr.com/ru/post/519564/


Здоровая критика и сравнения:
https://thegradient.pub/a-speech-to-text-practitioners-criticisms-of-industry-and-academia/

# Ссылки

Использованы и адаптированы материалы:


https://colab.research.google.com/github/Muennighoff/ytclipcc/blob/main/wav2vec_youtube_captions.ipynb

https://towardsdatascience.com/wav2vec-2-0-a-framework-for-self-supervised-learning-of-speech-representations-7d3728688cae

https://neurosys.com/blog/wav2vec-2-0-framework